In [1]:
import sys
import time
import zmq
from PyQt5 import sip
import pandas as pd

import configparser
import os
from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtCore import QSize, Qt
from PyQt5.QtWidgets import *

# 全局变量

In [2]:
AcctNameDict = {}

In [3]:
priceType ={}
priceType['1'] = "AnyPrice"
priceType['2'] = "LimitPrice"
priceType['u'] = "Unknown"
priceType

In [4]:
directionType ={}
directionType['0'] = "Buy"
directionType['1'] = "Sell"
directionType['u'] = "Unknown"
directionType

In [5]:
OpenCloseFlag ={}
OpenCloseFlag['0'] = "Open"
OpenCloseFlag['1'] = "Close"
OpenCloseFlag['3'] = "CloseToday"
OpenCloseFlag['4'] = "CloseYesterday"
OpenCloseFlag['u'] = "Unknown"
OpenCloseFlag

In [6]:
hedgeFlag = {}
hedgeFlag['u'] = "Unknown"
hedgeFlag['1'] = "Speculate"
hedgeFlag['2'] = "Arbitrage"
hedgeFlag

In [7]:
orderStatus ={}
orderStatus['1'] = "New"
orderStatus['2'] = "Accepted"
orderStatus['3'] = "Rejected"
orderStatus['4'] = "Cancelling"
orderStatus['5'] = "Canceled"
orderStatus['6'] = "PartTradedQueueing"
orderStatus['7'] = "PartTradedNotQueueing"
orderStatus['8'] = "AllTraded"
orderStatus['u'] = "Unknown"
orderStatus

# 相关函数

模拟数据

In [8]:
simulated_source=[
    ['IND_CONTRIBUTE,FLML,25,国防军工,-8.748'],
['IND_CONTRIBUTE,FLML,4,商贸零售,-8.696'],
['IND_CONTRIBUTE,FLML,18,有色金属,-7.311'],
['IND_CONTRIBUTE,FLML,11,交通运输,-6.241'],
['IND_CONTRIBUTE,FLML,2,医药,-6.093'],
['IND_CONTRIBUTE,FLML,21,计算机,-5.370'],
['IND_CONTRIBUTE,FLML,12,汽车,-4.818'],
['IND_CONTRIBUTE,FLML,1,房地产,-3.669'],
['IND_CONTRIBUTE,FLML,31,电力设备及新能源,-2.653'],
['IND_CONTRIBUTE,FLML,13,轻工制造,-2.628'],
['IND_CONTRIBUTE,FLML,6,建材,-2.330'],
['IND_CONTRIBUTE,FLML,7,家电,-1.953'],
['IND_CONTRIBUTE,FLML,14,电力及公用事业,-1.671'],
['IND_CONTRIBUTE,FLML,17,石油石化,-1.094'],
['IND_CONTRIBUTE,FLML,0,银行,-0.700'],
['IND_CONTRIBUTE,FLML,8,纺织服装,-0.014'],
['IND_CONTRIBUTE,FLML,27,钢铁,0.548'],
['IND_CONTRIBUTE,FLML,32,综合金融,0.784'],
['IND_CONTRIBUTE,FLML,23,煤炭,1.133'],
['IND_CONTRIBUTE,ML,31,电力设备及新能源,-0.011'],
['IND_CONTRIBUTE,ML,18,有色金属,0.008'],
['IND_CONTRIBUTE,ML,32,综合金融,0.175'],
['IND_CONTRIBUTE,FLOW,23,煤炭,-0.685'],
['IND_CONTRIBUTE,FLOW,17,石油石化,-0.645'],
['IND_CONTRIBUTE,FLOW,0,银行,-0.619'],
['IND_CONTRIBUTE,FLOW,27,钢铁,-0.280'],
['IND_CONTRIBUTE,FLOW,29,电子,-0.257'],
['IND_CONTRIBUTE,FLOW,25,国防军工,-0.090'],
['IND_CONTRIBUTE,FLOW,31,电力设备及新能源,0.211'],
['IND_CONTRIBUTE,FLOW,6,建材,0.357'],
['IND_CONTRIBUTE,FLOW,8,纺织服装,0.403'],
['IND_CONTRIBUTE,FLOW,7,家电,0.423'],
['IND_CONTRIBUTE,FLOW,28,传媒,0.994'],
['IND_CONTRIBUTE,FLOW,32,综合金融,1.192'],
['IND_CONTRIBUTE,FLOW,15,综合,1.546'],
['IND_CONTRIBUTE,FLOW,19,农林牧渔,3.212'],
['IND_CONTRIBUTE,FLOW,16,通信,3.972'],
['IND_CONTRIBUTE,FLOW,30,消费者服务,5.098'],
['IND_CONTRIBUTE,FLOW,9,食品饮料,5.807'],
['IND_CONTRIBUTE,FLOW,5,机械,6.463'],
['IND_CONTRIBUTE,ML,17,石油石化,0.263'],
['IND_CONTRIBUTE,ML,1,房地产,0.572'],
['IND_CONTRIBUTE,ML,22,基础化工,2.885'],
['IND_CONTRIBUTE,ML,8,纺织服装,3.693'],
['IND_CONTRIBUTE,ML,15,综合,3.927'],
['IND_CONTRIBUTE,ML,19,农林牧渔,4.482'],
['IND_CONTRIBUTE,ML,6,建材,4.736'],
['IND_CONTRIBUTE,ML,14,电力及公用事业,5.176'],
['IND_CONTRIBUTE,ML,5,机械,5.809'],
['IND_CONTRIBUTE,ML,26,非银行金融,7.059'],
['IND_CONTRIBUTE,ML,2,医药,7.718'],
['IND_CONTRIBUTE,ML,16,通信,8.013'],
['TOP_WINNER,FLML,600326.SH,7.523'],
['TOP_WINNER,FLML,002946.SZ,7.495'],
['TOP_WINNER,FLML,300031.SZ,7.442'],
['TOP_WINNER,FLML,603043.SH,7.311'],
['TOP_WINNER,FLML,002157.SZ,6.799'],
['TOP_WINNER,FLML,600739.SH,5.891'],
['TOP_WINNER,FLML,600273.SH,5.591'],
['TOP_WINNER,FLML,300315.SZ,5.268'],
['TOP_WINNER,FLML,002124.SZ,5.068'],
['TOP_WINNER,FLML,600597.SH,4.851'],
['TOP_WINNER,FLOW,600326.SH,7.533'],
['TOP_WINNER,FLOW,002946.SZ,7.504'],
['TOP_WINNER,FLOW,603043.SH,7.317'],
['TOP_WINNER,FLOW,600739.SH,5.904'],
['TOP_WINNER,FLOW,300394.SZ,5.840'],
['TOP_WINNER,FLOW,600273.SH,5.597'],
['TOP_WINNER,FLOW,002124.SZ,5.074'],
['TOP_WINNER,FLOW,300595.SZ,5.011'],
['TOP_WINNER,FLOW,600597.SH,4.705'],
['TOP_WINNER,FLOW,600984.SH,4.631'],
['TOP_WINNER,ML,300394.SZ,7.850'],
['TOP_WINNER,ML,600739.SH,5.904'],
['TOP_WINNER,ML,600737.SH,4.370'],
['TOP_WINNER,ML,002812.SZ,4.074'],
['TOP_WINNER,ML,601615.SH,3.977'],
['TOP_WINNER,ML,002773.SZ,3.942'],
['TOP_WINNER,ML,300529.SZ,3.250'],
['TOP_WINNER,ML,603588.SH,3.247'],
['TOP_WINNER,ML,300316.SZ,2.954'],
['TOP_WINNER,ML,002287.SZ,2.496'],
['TOP_LOSSER,FLML,603520.SH,-7.467'],
['TOP_LOSSER,FLML,600316.SH,-5.606'],
['TOP_LOSSER,FLML,600201.SH,-4.369'],
['TOP_LOSSER,FLML,300118.SZ,-4.086'],
['TOP_LOSSER,FLML,002183.SZ,-3.667'],
['TOP_LOSSER,FLML,000975.SZ,-3.038'],
['TOP_LOSSER,FLML,002155.SZ,-2.981'],
['TOP_LOSSER,FLML,002746.SZ,-2.884'],
['TOP_LOSSER,FLML,002557.SZ,-2.879'],
['TOP_LOSSER,FLML,300595.SZ,-2.875'],
['TOP_LOSSER,FLOW,603520.SH,-7.504'],
['TOP_LOSSER,FLOW,603718.SH,-7.492'],
['TOP_LOSSER,FLOW,600201.SH,-4.372'],
['TOP_LOSSER,FLOW,300118.SZ,-4.089'],
['TOP_LOSSER,FLOW,000975.SZ,-3.041'],
['TOP_LOSSER,FLOW,300315.SZ,-3.002'],
['TOP_LOSSER,FLOW,002155.SZ,-2.985'],
['TOP_LOSSER,FLOW,002557.SZ,-2.879'],
['TOP_LOSSER,FLOW,600298.SH,-2.735'],
['TOP_LOSSER,FLOW,600089.SH,-2.734'],
['TOP_LOSSER,ML,300101.SZ,-4.979'],
['TOP_LOSSER,ML,600201.SH,-4.372'],
['TOP_LOSSER,ML,300315.SZ,-3.002'],
['TOP_LOSSER,ML,300595.SZ,-2.902'],
['TOP_LOSSER,ML,002557.SZ,-2.879'],
['TOP_LOSSER,ML,600298.SH,-2.735'],
['TOP_LOSSER,ML,600859.SH,-2.565']
    
]

In [9]:
for i in range(len(simulated_source)):
    simulated_source[i]=simulated_source[i][0].split(',')

In [10]:
len(simulated_source)

109

In [11]:
def get_info():
    global simulated_source
    temp=simulated_source[0]
    simulated_source=simulated_source[1:]
    return temp

# 相关类

## updata类

In [12]:
class UpdateData_tab3(QtCore.QThread):
    requestChanged = QtCore.pyqtSignal(int, int, list)  # rowIndex, msgType,msg list

    def run(self):
        requestChanged = QtCore.pyqtSignal(int, int, list)
        context = zmq.Context()
        sock = context.socket(zmq.SUB)
        sock.setsockopt(zmq.SUBSCRIBE, b"IND_CONTRIBUTE")
        sock.setsockopt(zmq.SUBSCRIBE, b"TOP_WINNER")
        sock.setsockopt(zmq.SUBSCRIBE, b"TOP_LOSSER")
        sock.setsockopt(zmq.HEARTBEAT_IVL,     5000)
        sock.setsockopt(zmq.HEARTBEAT_TIMEOUT, 3000)
        #print("hello")
        sock.connect("tcp://192.168.0.32:19006")
		### DOUBLE RQID Problem
        
#         for i in range(108):
        while True:
#             real one
            msg = sock.recv()
            msgs = msg.decode("utf-8").split(",")
            #msgs= msg.split(',')
            #self.dataChanged.emit(2, 2, msgs[0])
            #print(msgs)
            if(msgs[0] == "IND_CONTRIBUTE"):
                self.requestChanged.emit(1,1, msgs[1:])

            elif(msgs[0] == "TOP_WINNER"):
                self.requestChanged.emit(2,2, msgs[1:])

            elif(msgs[0] == "TOP_LOSSER"):
                self.requestChanged.emit(2,3, msgs[1:])
            #test one:
#             msgs=get_info()
#             time.sleep(1)
            #print(msgs)
#             if(msgs[0] == "IND_CONTRIBUTE"):
#                 self.requestChanged.emit(1,1, msgs[1:])

#             elif(msgs[0] == "TOP_WINNER"):
#                 self.requestChanged.emit(2,2, msgs[1:])

#             elif(msgs[0] == "TOP_LOSSER"):
#                 self.requestChanged.emit(2,3, msgs[1:])


## Control_sys_Tab

In [13]:
class Control_sys_Tab(QTabWidget):
    def __init__(self,parent=None):
        self.RequestRowKey ={}
        self.OrderRowKey  ={}
        self.BatchRowKey  ={}   #To manage batch row index
        self.ErrorRowKey  ={}   #To manage error row index
        self.BuyBatchValue = {}
        self.SellBatchValue= {}
        self.BatchManagers  ={} # To manage value for each batch
        self.g_CurrRequestRow = 0
        self.g_CurrOrderRow  = 0
        self.g_CurrBatchRow  = 0
        self.g_CurrErrorRow  = 0
        
        #set config
        self.conf=configparser.ConfigParser()
        self.conf.read('./depend/config.txt')
        
        # tab3 use
        self.g_cat_row_dict={}
        self.cat_next_row=0
        
        self.g_winner_row_dict={}
        self.winner_next_row=0
        
        self.g_losser_row_dict={}
        self.losser_next_row=0
        
        
        super().__init__(parent)
        
        self.setObjectName("Control_system")
        self.resize(1800,985)
        self.setWindowTitle("实时监控系统")
        
        
        #创建3个选项卡小控件窗口
        self.tab1=QWidget()
        self.tab2=QWidget()
        self.tab3=QWidget()
        
        
        #将三个选项卡添加到顶层窗口中
        self.addTab(self.tab1, "交易界面")
        self.addTab(self.tab2, "PNL展示界面")
        self.addTab(self.tab3, "PNL定时发送")

        #每个选项卡自定义的内容
        self.tab1UI()
        self.tab2UI()
        self.tab3UI()
        
        
    
    def tab1UI(self):
        
        return
        
        
    def tab2UI(self):
        
        return 
    
    def tab3UI(self):
        
        #设置主布局
        layout=QHBoxLayout()
        
        sec_layout=QVBoxLayout()
        
        # 创建表格窗口1
        self.tableWidget4 = QtWidgets.QTableWidget()
        self.tableWidget4.setRowCount(40)
        fixed_list=self.conf.get('tab3','table4_lables').split(',')
        self.tableWidget4.setColumnCount(len(fixed_list))
        self.tableWidget4.setObjectName("tableWidget4")
        self.tableWidget4.setHorizontalHeaderLabels(fixed_list)
        for i in range (0,40):
            self.tableWidget4.setRowHeight(i,30)
        for i in range(0,len(fixed_list)):
            self.tableWidget4.horizontalHeaderItem(i).setTextAlignment(Qt.AlignHCenter)
            if (i == 1):
                self.tableWidget4.setColumnWidth(i, 200)
            else:
                self.tableWidget4.setColumnWidth(1, 100)
            
        
        # 表格窗口2
        self.tableWidget5 = QtWidgets.QTableWidget()
        self.tableWidget5.setRowCount(3000)
        fixed_list=self.conf.get('tab3','table5_lables').split(',')
        self.tableWidget5.setColumnCount(len(fixed_list))
        self.tableWidget5.setObjectName("tableWidget5")
        self.tableWidget5.setAutoFillBackground(True)
        self.tableWidget5.setHorizontalHeaderLabels(fixed_list)
        for i in range (0,3000):
            self.tableWidget5.setRowHeight(i,30)
        for i in range(0,len(fixed_list)):
            self.tableWidget5.horizontalHeaderItem(i).setTextAlignment(Qt.AlignHCenter)
            self.tableWidget5.setColumnWidth(i,180)
        
        # 表格窗口3
        self.tableWidget6 = QtWidgets.QTableWidget()

        self.tableWidget6.setRowCount(3000)
        fixed_list=self.conf.get('tab3','table6_lables').split(',')
        self.tableWidget6.setColumnCount(len(fixed_list))

        self.tableWidget6.setObjectName("tableWidget6")
        self.tableWidget6.setAutoFillBackground(True)
        
        self.tableWidget6.setHorizontalHeaderLabels(fixed_list)
        for i in range (0,3000):
            self.tableWidget6.setRowHeight(i,30)
        for i in range(0,len(fixed_list)):
            self.tableWidget6.horizontalHeaderItem(i).setTextAlignment(Qt.AlignHCenter)
            self.tableWidget6.setColumnWidth(i,180)


        self.pushButton3 = QtWidgets.QPushButton()
        #self.pushButton.setMaximumWidth(100)
        self.pushButton3.setObjectName("pushButton3")
        self.pushButton3.setText("开始运行")
        
        
        
        # 添加表单2进 子布局
        sec_layout.addWidget(self.tableWidget5)
        
        # 添加表单3进 子布局
        sec_layout.addWidget(self.tableWidget6)
        
        # 添加 按钮 进 子布局
        sec_layout.addWidget(self.pushButton3)
        
        # 添加表单1 进主布局，子布局进主布局
        layout.addWidget(self.tableWidget4)
        layout.addLayout(sec_layout)
        

        self.tab3.setLayout(layout)
        self.pushButton3.clicked.connect(self.slotStart_tab3)
    
    ##############################################################
#     PyQt5 中的pyQtslot 是python中的decorator，用其可以将一个method 定义为 槽
    
#     槽的传参方式 主要是直接传入一个 函数指针
    
    ##############################################################
    
    
    @QtCore.pyqtSlot()
    def slotStart_tab3(self):
        #按钮 暂停使用
        self.pushButton3.setEnabled(False) 
        # 开启一个新进程用来 更新数据
        self.update_data_thread3 = UpdateData_tab3(self) 
        self.update_data_thread3.requestChanged.connect(self.onRequestChanged_tab3)
        # 线程进入 准备阶段
        self.update_data_thread3.start()

    @QtCore.pyqtSlot(int, int, list)
    def onRequestChanged_tab3(self, row, msgType, text):
        # text 即为我们所需要的数据列
        #print(text)

        column = 0

        # message type
        
        if (msgType == 1):
            print(text)
            strategy_list=self.conf.get('tab3','table4_strategies').split(',')
            lable_list=self.conf.get('tab3','table4_lables').split(',')
            cat_id = self.g_cat_row_dict.get(text[1])
            if cat_id == None:
                cat_id = self.cat_next_row
                self.cat_next_row += 1
                self.g_cat_row_dict[text[1]] = cat_id
            ## cat_indx 和 cat_name
            for ele in text[1:(len(lable_list)-len(strategy_list))+1]:
                it = self.tableWidget4.item(cat_id, column)
                if it is None:
                    it = QtWidgets.QTableWidgetItem()
                    self.tableWidget4.setItem(cat_id, column, it)

                it.setText(ele)
                it.setTextAlignment(Qt.AlignCenter)
                column += 1

            # PNL

            if text[0] in strategy_list:
                col=strategy_list.index(text[0])
                col=len(lable_list)-len(strategy_list)+col
                it = self.tableWidget4.item(cat_id, col)
                if it is None:
                    it = QtWidgets.QTableWidgetItem()
                    self.tableWidget4.setItem(cat_id, col, it)
                it.setText(text[-1])
                it.setTextAlignment(Qt.AlignCenter)
                if float(text[-1]) <= -5:
                    it.setBackground(QtGui.QColor(144, 238, 144))
                if float(text[-1]) >= 5:
                    it.setBackground(QtGui.QColor(193, 210, 240))
#             if text[0] == 'FLOW':
#                 it = self.tableWidget4.item(cat_id, 5)
#                 if it is None:
#                     it = QtWidgets.QTableWidgetItem()
#                     self.tableWidget4.setItem(cat_id, 5, it)
#                 it.setText(text[-1])
#                 it.setTextAlignment(Qt.AlignCenter)
#                 if float(text[-1]) <= -5:
#                     it.setBackground(QtGui.QColor(144, 238, 144))
#                 if float(text[-1]) >= 5:
#                     it.setBackground(QtGui.QColor(193, 210, 240))
                
#             if text[0] == 'ML':
#                 it = self.tableWidget4.item(cat_id, 6)
#                 if it is None:
#                     it = QtWidgets.QTableWidgetItem()
#                     self.tableWidget4.setItem(cat_id, 6, it)
#                 it.setText(text[-1])
#                 it.setTextAlignment(Qt.AlignCenter)
#                 if float(text[-1]) <= -5:
#                     it.setBackground(QtGui.QColor(144, 238, 144))
#                 if float(text[-1]) >= 5:
#                     it.setBackground(QtGui.QColor(193, 210, 240))
                
#             if text[0] == 'FLML':
#                 it = self.tableWidget4.item(cat_id, 7)
#                 if it is None:
#                     it = QtWidgets.QTableWidgetItem()
#                     self.tableWidget4.setItem(cat_id, 7, it)
#                 it.setText(text[-1])
#                 it.setTextAlignment(Qt.AlignCenter)
#                 if float(text[-1]) <= -5:
#                     it.setBackground(QtGui.QColor(144, 238, 144))
#                 if float(text[-1]) >= 5:
#                     it.setBackground(QtGui.QColor(193, 210, 240))


        elif (msgType == 2):
            strategy_list=self.conf.get('tab3','table5_strategies').split(',')
            lable_list=self.conf.get('tab3','table5_lables').split(',')
            winner_id = self.g_winner_row_dict.get(text[1])
            if winner_id == None:
                winner_id = self.winner_next_row
                self.winner_next_row += 1
                self.g_winner_row_dict[text[1]] = winner_id
            ## winner_indx 和 winner_name
            for ele in text[1:(len(lable_list)-len(strategy_list))+1]:
                it = self.tableWidget5.item(winner_id, column)
                if it is None:
                    it = QtWidgets.QTableWidgetItem()
                    self.tableWidget5.setItem(winner_id, column, it)

                it.setText(ele)
                it.setTextAlignment(Qt.AlignCenter)
                column += 1

            # PNL
            
            if text[0] in strategy_list:
                col=strategy_list.index(text[0])
                col=len(lable_list)-len(strategy_list)+col
                it = self.tableWidget5.item(winner_id, col)
                if it is None:
                    it = QtWidgets.QTableWidgetItem()
                    self.tableWidget5.setItem(winner_id, col, it)
                it.setText(text[-2]+','+text[-1])
                it.setTextAlignment(Qt.AlignCenter)

                


        else:
            strategy_list=self.conf.get('tab3','table6_strategies').split(',')
            lable_list=self.conf.get('tab3','table6_lables').split(',')
            losser_id = self.g_losser_row_dict.get(text[1])
            if losser_id == None:
                losser_id = self.losser_next_row
                self.losser_next_row += 1
                self.g_losser_row_dict[text[1]] = losser_id
            ## losser_indx 和 losser_name
            for ele in text[1:(len(lable_list)-len(strategy_list))+1]:
                it = self.tableWidget6.item(losser_id, column)
                if it is None:
                    it = QtWidgets.QTableWidgetItem()
                    self.tableWidget6.setItem(losser_id, column, it)

                it.setText(ele)
                it.setTextAlignment(Qt.AlignCenter)
                column += 1

            # PNL
            strategy_list=self.conf.get('tab3','table6_strategies').split(',')
            lable_list=self.conf.get('tab3','table6_lables').split(',')
            if text[0] in strategy_list:
                col=strategy_list.index(text[0])
                col=len(lable_list)-len(strategy_list)+col
                it = self.tableWidget6.item(losser_id, col)
                if it is None:
                    it = QtWidgets.QTableWidgetItem()
                    self.tableWidget6.setItem(losser_id, col, it)
                it.setText(text[-2]+','+text[-1])
                it.setTextAlignment(Qt.AlignCenter)

# main

In [14]:
if __name__ == "__main__":

    app = QtWidgets.QApplication.instance()
    if app is None:
        app = QtWidgets.QApplication(sys.argv)
    w = Control_sys_Tab()
    w.show()
    sys.exit(app.exec_())

['FLML', '9', '食品饮料', '-0.014', '3.414', '2.183', '-6.203']
['FLML', '18', '有色金属', '-0.005', '3.362', '2.242', '-3.925']
['FLML', '26', '非银行金融', '0.008', '4.357', '-2.256', '-3.306']
['FLML', '6', '建材', '0.006', '1.609', '0.222', '-2.274']
['FLML', '5', '机械', '-0.008', '3.720', '2.244', '-2.262']
['FLML', '29', '电子', '0.012', '7.417', '2.237', '-1.485']
['FLML', '14', '电力及公用事业', '-0.002', '3.234', '-0.983', '-1.402']
['FLML', '7', '家电', '-0.012', '0.999', '0.085', '-1.274']
['FLML', '20', '建筑', '-0.007', '1.729', '0.746', '-1.251']
['FLML', '31', '电力设备及新能源', '0.006', '5.059', '1.247', '-1.101']
['FLML', '23', '煤炭', '-0.009', '0.687', '0.880', '-0.770']
['FLML', '8', '纺织服装', '-0.015', '0.371', '0.354', '-0.524']
['FLML', '27', '钢铁', '-0.006', '3.030', '-2.251', '-0.230']
['FLML', '32', '综合金融', '0.004', '0.426', '-0.426', '-0.182']
['FLML', '15', '综合', '-0.004', '0.977', '0.885', '0.011']
['FLML', '0', '银行', '-0.004', '1.249', '-1.234', '0.484']
['FLML', '2', '医药', '-0.007', '15.145', '-

['FLML', '9', '食品饮料', '-0.015', '3.414', '2.183', '-6.316']
['FLML', '18', '有色金属', '-0.007', '3.362', '2.242', '-3.651']
['FLML', '5', '机械', '-0.010', '3.720', '2.244', '-3.548']
['FLML', '26', '非银行金融', '0.005', '4.357', '-2.256', '-2.599']
['FLML', '6', '建材', '0.005', '1.609', '0.222', '-2.074']
['FLML', '20', '建筑', '-0.009', '1.729', '0.746', '-1.602']
['FLML', '29', '电子', '0.010', '7.417', '2.237', '-1.317']
['FLML', '7', '家电', '-0.013', '0.999', '0.085', '-1.258']
['FLML', '14', '电力及公用事业', '-0.003', '3.234', '-0.983', '-1.204']
['FLML', '31', '电力设备及新能源', '0.004', '5.059', '1.247', '-0.877']
['FLML', '23', '煤炭', '-0.011', '0.687', '0.880', '-0.681']
['FLML', '8', '纺织服装', '-0.017', '0.371', '0.354', '-0.309']
['FLML', '32', '综合金融', '0.004', '0.426', '-0.426', '-0.177']
['FLML', '15', '综合', '-0.005', '0.977', '0.885', '0.126']
['FLML', '27', '钢铁', '-0.007', '3.030', '-2.251', '0.132']
['FLML', '0', '银行', '-0.005', '1.249', '-1.234', '0.639']
['FLML', '30', '消费者服务', '-0.004', '0.551', 

SystemExit: 0

C:\Users\taosu\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
